In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF, StopWordsRemover,RegexTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString, StandardScaler
import shutil
import os
from pyspark.ml.stat import Correlation
import pandas as pd
import seaborn as sns

In [102]:
#spark.stop()

In [2]:
#open Spark Session
spark = SparkSession.builder.appName('prices_houses_v3').master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
#spark.sparkContext.addPyFile("sparkxgb.zip")

In [3]:
#read source I remove last 10 entries for predictions 
data=spark.read.csv("train.csv", inferSchema=True,sep=',',header=True)
datatest=spark.read.csv("test.csv", schema=data.schema,sep=',',header=True)

In [4]:
len(data.columns),len(datatest.columns)

(81, 81)

In [5]:
columnListS = [item[0] for item in data.dtypes if item[1].startswith('string')]
columnListI = [item[0] for item in data.dtypes if item[1].startswith('int')]
columnListS_T = [item[0] for item in datatest.dtypes if item[1].startswith('string')]
columnListI_T = [item[0] for item in datatest.dtypes if item[1].startswith('int')]

In [6]:
#datatest=datatest.fillna("NO_Value",subset=columnListS_T)
#datatest=datatest.fillna(0,subset=columnListI_T)
#data=data.fillna("NO_Value",subset=columnListS)

In [7]:
features=columnListI[1:34]
featurest=columnListI_T[1:34]


In [8]:

lr_data = data.select(sf.col("SalePrice").alias("label"), *columnListI).dropna()

columns = lr_data.columns

In [9]:
lr_datat = datatest.select(sf.col("SalePrice").alias("label"), *columnListI_T)
columnst = lr_datat.columns

In [19]:
type(columnst)

list

In [10]:
lr_datat=lr_datat.fillna(0)

In [11]:
lr_datat.show(n=1, truncate=False)

+-----+----+----------+-------+-----------+-----------+---------+------------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+---------+
|label|Id  |MSSubClass|LotArea|OverallQual|OverallCond|YearBuilt|YearRemodAdd|BsmtFinSF1|BsmtFinSF2|BsmtUnfSF|TotalBsmtSF|1stFlrSF|2ndFlrSF|LowQualFinSF|GrLivArea|BsmtFullBath|BsmtHalfBath|FullBath|HalfBath|BedroomAbvGr|KitchenAbvGr|TotRmsAbvGrd|Fireplaces|GarageCars|GarageArea|WoodDeckSF|OpenPorchSF|EnclosedPorch|3SsnPorch|ScreenPorch|PoolArea|MiscVal|MoSold|YrSold|SalePrice|
+-----+----+----------+-------+-----------+-----------+---------+------------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-------

In [12]:
resumen2=lr_data.toPandas()
resumen2.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
label,1460.0,180921.195890,79442.502883,34900.0,129975.00,163000.0,214000.00,755000.0
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0
BsmtFinSF2,1460.0,46.549315,161.319273,0.0,0.00,0.0,0.00,1474.0


In [13]:
corr=resumen2.corr()

In [14]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [15]:
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [20]:
features_extra=['LotArea','OverallQual','YearBuilt','YearRemodAdd','BsmtFinSF1','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','BsmtFullBath','FullBath','HalfBath','TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea']

In [21]:
type(features_extra)

list

In [22]:
vector = VectorAssembler(inputCols=features_extra, outputCol="features")
scaler = StandardScaler(withMean = True, withStd = True,inputCol="features", outputCol="scaled_features")

stages = [vector, scaler]

pipe = Pipeline(stages=stages)

# we'll be using this data frame
data_out = pipe.fit(lr_data).transform(lr_data).select('id','scaled_features','label')

In [23]:
#lr_datat TEST DATA

vector2 = VectorAssembler(inputCols=features_extra, outputCol="features")
scaler2 = StandardScaler(withMean = True, withStd = True,inputCol="features", outputCol="scaled_features")

stages = [vector2, scaler2]

pipe2 = Pipeline(stages=stages)

# we'll be using this data frame
data_out_t = pipe2.fit(lr_datat).transform(lr_datat).select('id','scaled_features','label')

In [24]:
data_out=data_out.withColumnRenamed('scaled_features','features')

In [25]:
data_out_t=data_out_t.withColumnRenamed('scaled_features','features')

In [26]:
train_data,test_data = data_out.randomSplit([0.7,0.3])

In [27]:
data_out.show()

+---+--------------------+------+
| id|            features| label|
+---+--------------------+------+
|  1|[-0.2070707566820...|208500|
|  2|[-0.0918548989935...|181500|
|  3|[0.07345480986380...|223500|
|  4|[-0.0968642841104...|140000|
|  5|[0.37501979390060...|250000|
|  6|[0.36049257706162...|143000|
|  7|[-0.0433640510620...|307000|
|  8|[-0.0135081157654...|200000|
|  9|[-0.4405081031291...|129900|
| 10|[-0.3102640900899...|118000|
| 11|[0.06844542474691...|129500|
| 12|[0.14098132123949...|345000|
| 13|[0.24557728248016...|144000|
| 14|[0.01354256386579...|279500|
| 15|[0.04039286809233...|157000|
| 16|[-0.4405081031291...|132000|
| 17|[0.07255312054276...|149000|
| 18|[0.02746865449075...| 90000|
| 19|[0.31841374207974...|159000|
| 20|[-0.2962378117627...|139000|
+---+--------------------+------+
only showing top 20 rows



In [28]:
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator

In [29]:
lr = LinearRegression()

In [30]:
lrModel = lr.fit(train_data)

In [31]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [6124.35175593,26950.8989657,7971.97557579,6514.709574,4561.69756303,3243.20294519,24086.2911021,23931.1223819,-9443.21100503,3896.99062561,-942.753737262,-429.432272213,3968.95482744,5240.49560315,7079.39461983,3003.19327769] Intercept: 180920.92978486384


In [32]:
test_results = lrModel.evaluate(test_data)

In [33]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 33067.63164092151
MSE: 1093468262.3396733
R2: 0.8312901566359373


In [34]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [35]:
gbt = GBTRegressor()

In [36]:
model = gbt.fit(train_data)

In [37]:
predictions = model.transform(test_data)

In [38]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.857163


In [39]:
predictions2 = model.transform(data_out_t)

In [40]:
output=predictions2.select('id','prediction')

In [41]:
output.write.csv('output4.csv')

In [42]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [43]:
rf = RandomForestRegressor()

In [44]:
modelrf = rf.fit(train_data)

In [45]:
predictionsrf = modelrf.transform(test_data)

In [46]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2")
rmse = evaluator.evaluate(predictionsrf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.842659


In [47]:
predictions3 = modelrf.transform(data_out_t)

In [48]:
output=predictions3.select('id','prediction')

In [50]:
output.write.csv('output5.csv')